<h2>Exploring the protein-protein network </h2>

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os

import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics

from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

/Users/ravanv/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/ravanv/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#bio-decagon-ppi:Protein-protein interaction network
#bio-decagon-targets: Drug-target protein associations
#bio-decagon-targets-all: Drug-target protein associations culled from several curated databases
#bio-decagon-combo:Polypharmacy side effects in the form of (drug A, side effect type, drug B) triples
#bio-decagon-effectcategories: Side effect categories
#bio-decagon-mono:Side effects of individual drugs in the form of (drug A, side effect type) tuples

<h1>Datasets from the paper:</h1>
<h2>The protein-prtoein network:</h2>
number of proteins = 19,085, number of physical interactions = 719,402
<h2>Final Network:</h2>
<h4>Number of protiens = 19,085 (paper) ....... Number of protiens = 19,081 (protein-protein network (ppi))</h4>
<h4>Number of protien-protien edges= 715,612 (paper)........ Number of protien-protien edges= 715,612 (ppi)</h4>


In [3]:
protein_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-ppi.csv", sep=',',header = 0)
print("size of the protein-protien network:", protein_protein_data.shape)


size of the protein-protien network: (715612, 2)


In [4]:
column_list = list(protein_protein_data)
column_list

['Gene 1', 'Gene 2']

In [5]:
list_gene1 = []
prev_gene = []
for i in range(0,protein_protein_data.shape[0]):
    if protein_protein_data.loc[i][0] not in prev_gene:
        list_gene1.append(protein_protein_data.loc[i][0])
        prev_gene.append(protein_protein_data.loc[i][0])

In [6]:
len(list_gene1)

17544

In [7]:
list_gene2 = []
prev_gene = []
for i in range(0,protein_protein_data.shape[0]):
    if protein_protein_data.loc[i][1] not in prev_gene:
        list_gene2.append(protein_protein_data.loc[i][1])
        prev_gene.append(protein_protein_data.loc[i][1])

In [8]:
len(list_gene2)

17472

In [9]:
def intersection(list1, list2):
    lst3 = [value for value in list1 if value in list2]
    return lst3
def onlylist(lst,intersect):
    lst3 = [value for value in lst if value not in intersect]
    return lst3

In [10]:
intersect = intersection(list_gene2,list_gene1)
list1_only = onlylist(list_gene1,intersect)
list2_only = onlylist(list_gene2,intersect)

In [11]:
list_proteins = intersect + list1_only + list2_only

In [12]:
print("number of protiens in the first column = ", len(list1_only))
print("number of protiens in the second column = ", len(list2_only))
print("number of protiens that exist in both columns = ",len(intersect))
print("total number of protiens  = ",len(list_proteins))

number of protiens in the first column =  1609
number of protiens in the second column =  1537
number of protiens that exist in both columns =  15935
total number of protiens  =  19081
